In [1]:
import numpy as np
 
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
 
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model # This is the functional API
 
inception =InceptionV3(weights='imagenet', include_top=False) # 모델에서 Fully Connected layer 부분을 제거해주는 역할

87910968/87910968 [==============================] - 1s 0us/step


In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os

base_dir = '/content/drive/MyDrive/apple'
base_apple_banana_orange_dir = '/content/drive/MyDrive/apple_banana_orange'

train_dir = os.path.join(base_apple_banana_orange_dir, 'train')
train_fresh_dir = os.path.join(train_dir, 'fresh')
train_rotten_dir = os.path.join(train_dir, 'rotten')

test_apple_dir = os.path.join(base_dir, 'test_apple')
test_apple_fresh_dir = os.path.join(test_apple_dir, 'fresh')
test_apple_rotten_dir = os.path.join(test_apple_dir, 'rotten')

test_banana_dir = os.path.join(base_dir, 'test_banana')
test_banana_fresh_dir = os.path.join(test_banana_dir, 'fresh')
test_banana_rotten_dir = os.path.join(test_banana_dir, 'rotten')

test_orange_dir = os.path.join(base_dir, 'test_orange')
test_orange_fresh_dir = os.path.join(test_orange_dir, 'fresh')
test_orange_rotten_dir = os.path.join(test_orange_dir, 'rotten')

In [4]:
train_fresh_fnames = os.listdir(train_fresh_dir)
train_fresh_fnames.sort()
print(train_fresh_fnames[:10])

train_rotten_fnames = os.listdir(train_rotten_dir)
train_rotten_fnames.sort()
print(train_rotten_fnames[:10])

['Screen Shot 2018-06-08 at 4.59.36 PM.png', 'Screen Shot 2018-06-08 at 4.59.49 PM.png', 'Screen Shot 2018-06-08 at 4.59.57 PM.png', 'Screen Shot 2018-06-08 at 5.00.03 PM.png', 'Screen Shot 2018-06-08 at 5.00.12 PM.png', 'Screen Shot 2018-06-08 at 5.00.18 PM.png', 'Screen Shot 2018-06-08 at 5.00.26 PM.png', 'Screen Shot 2018-06-08 at 5.00.35 PM.png', 'Screen Shot 2018-06-08 at 5.00.43 PM.png', 'Screen Shot 2018-06-08 at 5.00.50 PM.png']
['Screen Shot 2018-06-07 at 2.15.20 PM.png', 'Screen Shot 2018-06-07 at 2.15.50 PM.png', 'Screen Shot 2018-06-07 at 2.16.18 PM.png', 'Screen Shot 2018-06-07 at 2.16.41 PM.png', 'Screen Shot 2018-06-07 at 2.17.15 PM.png', 'Screen Shot 2018-06-07 at 2.17.25 PM.png', 'Screen Shot 2018-06-07 at 2.18.13 PM.png', 'Screen Shot 2018-06-07 at 2.18.57 PM.png', 'Screen Shot 2018-06-07 at 2.19.15 PM.png', 'Screen Shot 2018-06-07 at 2.19.37 PM.png']


In [5]:
print('total training fresh images:', len(os.listdir(train_fresh_dir)))
print('total training rotten images:', len(os.listdir(train_rotten_dir)))
print('total test_apple fresh images:', len(os.listdir(test_apple_fresh_dir)))
print('total test_apple rotten images:', len(os.listdir(test_apple_rotten_dir)))
print('total test_banana fresh images:', len(os.listdir(test_banana_fresh_dir)))
print('total test_banana rotten images:', len(os.listdir(test_banana_rotten_dir)))
print('total test_orange fresh images:', len(os.listdir(test_orange_fresh_dir)))
print('total test_orange rotten images:', len(os.listdir(test_orange_rotten_dir)))

total training fresh images: 4740
total training rotten images: 6161
total test_apple fresh images: 395
total test_apple rotten images: 601
total test_banana fresh images: 381
total test_banana rotten images: 530
total test_orange fresh images: 388
total test_orange rotten images: 403


In [6]:
from tensorflow.keras import layers
img_input = layers.Input(shape=(224*224*3))

In [7]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_apple_datagen = ImageDataGenerator(rescale=1./255)
test_banana_datagen = ImageDataGenerator(rescale=1./255)
test_orange_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,  
        target_size=(224, 224),
        batch_size=256,
        classes = ['fresh','rotten'],
        class_mode="binary")

test_apple_generator = test_apple_datagen.flow_from_directory(
        test_apple_dir,
        target_size=(224, 224),
        batch_size=256,
        classes = ['fresh', 'rotten'],
        class_mode="binary")


test_banana_generator = test_banana_datagen.flow_from_directory(
        test_banana_dir,
        target_size=(224, 224),
        batch_size=256,
        classes = ['fresh', 'rotten'],
        class_mode="binary")

test_orange_generator = test_orange_datagen.flow_from_directory(
        test_orange_dir,
        target_size=(224, 224),
        batch_size=256,
        classes = ['fresh', 'rotten'],
        class_mode="binary")


# 전체데이터 : 배치사이즈 수정 필요
# resnet50 input image size ; 224 x 224 x 3

Found 10901 images belonging to 2 classes.
Found 996 images belonging to 2 classes.
Found 911 images belonging to 2 classes.
Found 791 images belonging to 2 classes.


In [8]:
for layer in inception.layers:
    layer.trainable = False
# ResNet50 레이어들의 파라미터를 학습하지 않도록 설정합니다.
# 이렇게 설정된 매개 변수는 역전파를 통해 오차 정보가 전파 되더라도 파라미터가 업데이트 되지 않습니다.

In [9]:
x = inception.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inception.input, predictions)

In [10]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [11]:
history = model.fit(
      train_generator, 
      epochs=30,
      verbose=1)

Epoch 1/30
43/43 [==============================] - 1682s 39s/step - loss: 0.1978 - accuracy: 0.9155
Epoch 2/30
43/43 [==============================] - 63s 1s/step - loss: 0.0723 - accuracy: 0.9739
Epoch 3/30
43/43 [==============================] - 61s 1s/step - loss: 0.0444 - accuracy: 0.9845
Epoch 4/30
43/43 [==============================] - 61s 1s/step - loss: 0.0330 - accuracy: 0.9902
Epoch 5/30
43/43 [==============================] - 61s 1s/step - loss: 0.0270 - accuracy: 0.9919
Epoch 6/30
43/43 [==============================] - 60s 1s/step - loss: 0.0175 - accuracy: 0.9965
Epoch 7/30
43/43 [==============================] - 61s 1s/step - loss: 0.0134 - accuracy: 0.9978
Epoch 8/30
43/43 [==============================] - 60s 1s/step - loss: 0.0106 - accuracy: 0.9985
Epoch 9/30
43/43 [==============================] - 60s 1s/step - loss: 0.0101 - accuracy: 0.9983
Epoch 10/30
43/43 [==============================] - 59s 1s/step - loss: 0.0070 - accuracy: 0.9994
Epoch 11/30
43/4

In [12]:
loss_and_metrics_apple = model.evaluate(test_apple_generator)
print('loss_and_metrics_apple : ' + str(loss_and_metrics_apple))

4/4 [==============================] - 221s 73s/step - loss: 0.0094 - accuracy: 0.9980
loss_and_metrics_apple : [0.009382816031575203, 0.9979919791221619]


In [13]:
loss_and_metrics_banana = model.evaluate(test_banana_generator)
print('loss_and_metrics_banana : ' + str(loss_and_metrics_banana))

4/4 [==============================] - 190s 63s/step - loss: 0.0033 - accuracy: 0.9989
loss_and_metrics_banana : [0.0033220830373466015, 0.9989023208618164]


In [14]:
loss_and_metrics_orange = model.evaluate(test_orange_generator)
print('loss_and_metrics_orange : ' + str(loss_and_metrics_orange))

4/4 [==============================] - 150s 50s/step - loss: 0.0153 - accuracy: 0.9937
loss_and_metrics_orange : [0.015255576930940151, 0.993678867816925]


In [15]:
# 모델 저장하기
from keras.models import load_model
model.save('apple_banana_orange_InceptionV3_Dense256_epoch30.h5')

# 2. 모델 불러오기
# from keras.models import load_model
# model = load_model('모델파일명.h5')